Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

### Содержание
### [1. Подготовка](#section_id1)
### [2. Обучение](#section_id2)
### [3. Выводы](#section_id3)

<a id='section_id1'></a>

# 1. Подготовка

In [1]:
import pandas as pd
from sklearn.metrics import f1_score
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
import re
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Датафрейм состоит только из двух столбцов: 1-ый - тексты, 2-ой целевой признак. Перед тем как проводить обучение модели, необходимо провести лемматизацию, удалить ненужные слова (как правило это союзы предлоги и всякие частички), посчитаем вел-ну TF-IDF

In [4]:
#Преобразуем столбец text в список текстов. Переведём тексты в стандартный для Python формат: кодировку Unicode (U)
corpus = df['text'].values.astype('U')

In [5]:
new_clear = []
new_lemm = []

In [6]:
#проведем чистку 
def clear_text(text):
    for i in range(len(text)):
        clear_text = re.sub(r'[^a-zA-Z ]', ' ', text[i])
        new_clear.append(" ".join(clear_text.split()))
        
    return new_clear

In [7]:
#проведем лемматизацию слов
def lemmatize(text):
    #print(text)
    for i in text:
        word_list = nltk.word_tokenize(i)
        lemmatized_output = " ".join([lemmatizer.lemmatize(j) for j in word_list])
        new_lemm.append(lemmatized_output)
    
    return new_lemm

In [8]:
lemmatizer = WordNetLemmatizer()

In [9]:
df['lemm_text'] = clear_text(lemmatize(corpus))

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id='section_id2'></a>

# 2. Обучение

In [11]:
model = LogisticRegression(random_state = 12345, C=2.5, max_iter=1000)

In [12]:
features = df['lemm_text']
target = df['toxic']

In [13]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.2)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5)

In [14]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(127656,)
(127656,)
(15957,)
(15957,)
(15958,)
(15958,)


In [15]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(features_train)

In [16]:
%%time
model.fit(tf_idf_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 7.44 s, sys: 3.96 s, total: 11.4 s
Wall time: 11.4 s


LogisticRegression(C=2.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
tf_idf_valid = count_tf_idf.transform(features_valid)
predictions = model.predict(tf_idf_valid)
print(f1_score(target_valid, predictions))

0.7581512002866356


In [18]:
tf_idf_test = count_tf_idf.transform(features_test)
predictions = model.predict(tf_idf_test)
print(f1_score(target_test, predictions))

0.7613793103448275


Валидационная выборка модели "Логистическая регрессия" показала f1-метрику = 0,756, а тестовая даже лучше 0,762. Данная метрика нас полностью удовлетворяет. Но ради интереса посмотрим, как поведет себя  метрика если мы исключим дисбаланс классов целевых признаков.

In [19]:
#напишем функцию дисбаланса
def class_balance (features_train, target_train):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    return (features_zeros, features_ones, target_zeros, target_ones)

In [20]:
features_zeros, features_ones, target_zeros, target_ones = class_balance(features_train, target_train)
print(features_zeros.shape)
print(features_ones.shape)
print(target_zeros.shape)
print(target_ones.shape)

(114742,)
(12914,)
(114742,)
(12914,)


In [21]:
#напишем функцию, которая увеличивает положительную выборку (новые не создаются, а просто дублируются существующие
#и перемешиваются)
def upsample(features, target, repeat):
    features_zeros, features_ones, target_zeros, target_ones = class_balance (features, target)
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    #print(target_upsampled.value_counts())
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=123456)
    return features_upsampled, target_upsampled

In [22]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)#так как отношение 1/10
print(features_upsampled.shape)
print(target_upsampled.shape)

(243882,)
(243882,)


In [23]:
count_tf_idf_sample = TfidfVectorizer(stop_words=stopwords)
tf_idf_train_sample = count_tf_idf_sample.fit_transform(features_upsampled)

In [24]:
model.fit(tf_idf_train_sample, target_upsampled)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=2.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
tf_idf_valid = count_tf_idf_sample.transform(features_valid)
predictions = model.predict(tf_idf_valid)
print(f1_score(target_valid, predictions))

0.7585235277542969


In [26]:
tf_idf_test = count_tf_idf_sample.transform(features_test)
predictions = model.predict(tf_idf_test)
print(f1_score(target_test, predictions))

0.7550585729499468


При устранении дисбалансов класса путем увеличения положительных классов в 10 раз (мы просто дублировали, нового ничего не добавляли), валидационная метрика стала лучше 0,765, и на тестовой выборке качество 0,763.

Проверим модель случайный лес

In [ ]:
%%time
model = RandomForestClassifier(random_state = 12345, n_estimators = 100, max_depth = 50)
model.fit(tf_idf_train_sample, target_upsampled)
predictions_valid = model.predict(tf_idf_valid)
print(f'max_depth = 100:', end=' ')
print(f'n_estimators = 100:', end=' ')
print('f1 score =', f1_score(target_valid, predictions_valid))

Как видим, модель случайный лес очень долго обучается и f1-метрика не соответствует заданным требованиям. (Используя более высокие значения гиперпараметров, может и можно добиться нужной метрики но это займет очень большое кол-во времени) 

<a id='section_id3'></a>

# 3. Выводы

Итак, нам был предоставлен датафрейм, состоящий из двух признаков: текст и признак определяющий токсичные комментарии. Перед началом обучения, была проведена очистка текстов от цифр, знаков препинания, подчеркиваний и т.д. После чего мы провели лемматизацию текста (привели в исходное состояние текста). Далее мы еще избавились от левых слов (типа союза предлога и т.д.). Использовалась модель логистическая регрессия, случайный лес и я честно пытался попробовать модель градиентного бустинга и BERT (но кернел все время умирал, что удаленно, что и локально в юпитере). Перед началом обучения мы перевели текст в векторное представление путев TF-IDF. Также наша выборка была разбита на 3 датасета (трейн, валид и тест). В итоге логистическая регрессия на валидационной и тестовой выборках показали f1-метрику выше 0,75, что соответствует заданию. Также я решил попробовать увеличить метрику путем исключения дисбаланса классов путем увеличения положительных значений (1 - положительная, 0 - отрицательная). В итоге качество улучшилось, но не намного, что не в прочем не имеет смысла усложнять работы и можно обойтись без операций дисбаланса.